# Molecular graphs

First, we need to load/install the package we will need to run this Python notebook.

In [ ]:
from rdkit.Chem import MolFromSmiles, MolToSmiles, rdmolops
from rdkit.Chem import rdDistGeom, AllChem
import networkx as nx
import matplotlib.pyplot as plt

Next, we need to load in the molecule that we are interested in. We will use a SMILES string to do this, which can be interpreted by RDKit using the below code. We will define a variable `smi` for our SMILES string.

In [ ]:
smi = "OCC1CO1"

Now we need to convert it to an RDKit mol object so we can use it to generate our graph.

In [ ]:
mol = MolFromSmiles(smi)

The first type of graph we will generate is an **adjacency matrix**. Remember, this describes the internal connectivity of a molecule.

In [ ]:
A = rdmolops.GetAdjacencyMatrix(mol)
print(A)

The above representation is printed as a mathematical object, but it can be more useful to draw the actual graph from this adjacency matrix. We will construct the graph below using the NetworkX package.

*What do you notice about the graph you have drawn and the information that it contains?*

In [ ]:
# Create a NetworkX graph from the adjacency matrix
G = nx.from_numpy_array(A)

# Draw the network
plt.figure(figsize=(5, 5))
nx.draw(G, with_labels=True, node_color='lightblue', 
    node_size=500, font_size=16, font_weight='bold')
plt.title("Molecular Graph Network")
plt.show()

Next, we will encode some distance information into the matrix, we can use either the **topological distance** or the **Euclidean distance** between atoms. This gives us the **distance matrix**. Like above, we can also construct the graph from this distance matrices.

First, we will look at the topological distance matrix:

In [ ]:
distance_matrix = rdmolops.GetDistanceMatrix(mol)
print(distance_matrix)

In [ ]:
# Create a new NetworkX graph with distance information as edge weights
G_weighted = nx.Graph()

# Add nodes
for i in range(len(distance_matrix)):
    G_weighted.add_node(i)

# Add edges with distances as weights (only for directly connected atoms)
for i in range(len(A)):
    for j in range(i+1, len(A)):
        if A[i][j] == 1:  # Only add edges where atoms are directly connected
            G_weighted.add_edge(i, j, weight=distance_matrix[i][j])

# Draw the weighted network
plt.figure(figsize=(5, 5))
pos = nx.spring_layout(G_weighted)
nx.draw(G_weighted, pos, with_labels=True, node_color='lightgreen', 
        node_size=700, font_size=16, font_weight='bold')

# Draw edge labels with distances
edge_labels = nx.get_edge_attributes(G_weighted, 'weight')
edge_labels = {k: f'{v:.2f}' for k, v in edge_labels.items()}
nx.draw_networkx_edge_labels(G_weighted, pos, edge_labels)

plt.title("Molecular Graph with Topological Distance Information")
plt.show()

*What do you notice about this graph?*

Next, we will look at the distance matrix derived from Euclidean distances.

In [ ]:
distance_matrix = rdDistGeom.GetMoleculeBoundsMatrix(mol)
print(distance_matrix)

In [ ]:
# Create a new NetworkX graph with distance information as edge weights
G_weighted = nx.Graph()

# Add nodes
for i in range(len(distance_matrix)):
    G_weighted.add_node(i)

# Add edges with distances as weights (only for directly connected atoms)
for i in range(len(A)):
    for j in range(i+1, len(A)):
        if A[i][j] == 1:  # Only add edges where atoms are directly connected
            G_weighted.add_edge(i, j, weight=distance_matrix[i][j])

# Draw the weighted network
plt.figure(figsize=(5, 5))
pos = nx.spring_layout(G_weighted)
nx.draw(G_weighted, pos, with_labels=True, node_color='lightgreen', 
        node_size=700, font_size=16, font_weight='bold')

# Draw edge labels with distances
edge_labels = nx.get_edge_attributes(G_weighted, 'weight')
edge_labels = {k: f'{v:.2f}' for k, v in edge_labels.items()}
nx.draw_networkx_edge_labels(G_weighted, pos, edge_labels)

plt.title("Molecular Graph with Euclidean Distance Information")
plt.show()

This is perhaps more useful, but the distances actually come from 2D connectivity. We can make this graph more useful by getting the 3D distance information.

First, we need RDKit to convert our 2D mol object to a 3D one (for simplicity, we will ignore the hydrogen atoms):

In [ ]:
AllChem.EmbedMolecule(mol)

Now we need to generate the distance matrix:

In [ ]:
D = AllChem.Get3DDistanceMatrix(mol)
print(D)

Then we can generate the graph structure. *What differences do you notice comparing this graph to the 2D one above?*

In [ ]:
# Create a new NetworkX graph with distance information as edge weights
G_weighted = nx.Graph()

# Add nodes
for i in range(len(D)):
    G_weighted.add_node(i)

# Add edges with distances as weights (only for directly connected atoms)
for i in range(len(A)):
    for j in range(i+1, len(A)):
        if A[i][j] == 1:  # Only add edges where atoms are directly connected
            G_weighted.add_edge(i, j, weight=D[i][j])

# Draw the weighted network
plt.figure(figsize=(5, 5))
pos = nx.spring_layout(G_weighted)
nx.draw(G_weighted, pos, with_labels=True, node_color='lightgreen', 
        node_size=700, font_size=16, font_weight='bold')

# Draw edge labels with distances
edge_labels = nx.get_edge_attributes(G_weighted, 'weight')
edge_labels = {k: f'{v:.2f}' for k, v in edge_labels.items()}
nx.draw_networkx_edge_labels(G_weighted, pos, edge_labels)

plt.title("3D Molecular Graph with Euclidean Distance Information")
plt.show()

Finally, we can generate a **weighted graph** by combining the adjacency matrix `A` and the 3D distance matrix `D`:

In [ ]:
weighted_graph = A * D
print(weighted_graph)

In [ ]:
# Create a new NetworkX graph with distance information as edge weights
G_weighted = nx.Graph()

# Add nodes
for i in range(len(weighted_graph)):
    G_weighted.add_node(i)

# Add edges with distances as weights (only for directly connected atoms)
for i in range(len(A)):
    for j in range(i+1, len(A)):
        if A[i][j] == 1:  # Only add edges where atoms are directly connected
            G_weighted.add_edge(i, j, weight=weighted_graph[i][j])

# Draw the weighted network
plt.figure(figsize=(5, 5))
pos = nx.spring_layout(G_weighted)
nx.draw(G_weighted, pos, with_labels=True, node_color='lightgreen', 
        node_size=700, font_size=16, font_weight='bold')

# Draw edge labels with distances
edge_labels = nx.get_edge_attributes(G_weighted, 'weight')
edge_labels = {k: f'{v:.2f}' for k, v in edge_labels.items()}
nx.draw_networkx_edge_labels(G_weighted, pos, edge_labels)

plt.title("3D Molecular Graph with Euclidean Distance Information")
plt.show()

*What differences do you notice comparing the 3D molecular graph and the weighted graph we just made?*